In [284]:
import pandas
from typing import Sequence
from 

In [352]:
ds = pandas.read_csv('stats/2021-2022.csv', header=0, encoding = 'unicode_escape')

In [353]:
ds = ds[ds["S%"] != '--']
ds = ds[ds["P"] != 0]
ds = ds[ds["GP"] > 4]
ds["S%"] = ds["S%"].astype(float) / 100

In [354]:
def convert_toi_to_seconds(toi):
    min_seconds = toi.split(':')
    return int(min_seconds[0]) * 60 + int(min_seconds[1])

ds["TOI%/GP"] = ds["TOI/GP"].apply(convert_toi_to_seconds) / 3600

# bin number of played games over total number of games (accounting for shortened seasons)
ds['GP%-B'] = pandas.cut(ds["GP"] / ds["GP"].max(), [0, 0.25, 0.5, 0.75, 1.], labels=range(0,4))

In [355]:
teams_abv = list(filter(lambda x: ',' not in x, ds["Team"].unique()))

In [356]:
ds["EVA"] = ds["EVP"] - ds["EVG"]
ds["PPA"] = ds["PPP"] - ds["PPG"]
ds["SHA"] = ds["SHP"] - ds["SHG"]

In [357]:
ds["Center"] = (ds["Pos"] == 'C').astype(int)
ds["Winger"] = (ds["Pos"].isin(['L','R'])).astype(int)
ds["Def"] = (ds["Pos"] == 'D').astype(int)

ds["Left"] = (ds["Shoots"] == 'L').astype(int)
ds["Right"] = (ds["Shoots"] == 'R').astype(int)

In [358]:
def drop_columns(df: pandas.DataFrame, columns: Sequence[str]) -> pandas.DataFrame:
    for col in columns:
        df = df.drop(col, axis=1)

    return df

def normalize_per_gp(df: pandas.DataFrame, columns: Sequence[str]) -> pandas.DataFrame:
    games_played: int = df["GP"].astype(float)

    for col in columns:
        df[f"{col}/GP"] = df[col].astype(float) / games_played

    df = drop_columns(df, columns)
    
    return df

ds = normalize_per_gp(ds, ["EVA", "EVG", "PPA", "PPG", "SHA", "SHG", "S", "PIM", "+/-"])

In [359]:
def drop_columns(df: pandas.DataFrame, columns: Sequence[str]) -> pandas.DataFrame:
    for col in columns:
        df = df.drop(col, axis=1)

    return df

ds = drop_columns(ds, ["Season", "#", "FOW%", "G", "A", "P", "Shoots", "Pos", "EVP", "SHP", "PPP", "P/GP", "OTG", "GWG", "Team", "TOI/GP", "GP"])

In [360]:
ds.head(5)

,Player,S%,TOI%/GP,GP%-B,Center,Winger,Def,Left,Right,EVA/GP,EVG/GP,PPA/GP,PPG/GP,SHA/GP,SHG/GP,S/GP,PIM/GP,+/-/GP
0,Leon Draisaitl,0.250,0.382778,3,1,0,0,1,0,0.615385,0.346154,0.230769,0.500000,0.0,0.038462,3.538462,0.692308,0.500000
1,Connor McDavid,0.163,0.375278,3,1,0,0,1,0,0.500000,0.423077,0.615385,0.192308,0.0,0.000000,3.769231,0.961538,0.192308
2,Alex Ovechkin,0.165,0.365556,3,0,1,0,0,1,0.678571,0.535714,0.178571,0.142857,0.0,0.035714,4.321429,0.142857,0.678571
3,Nazem Kadri,0.159,0.303056,2,1,0,0,1,0,0.727273,0.272727,0.318182,0.227273,0.0,0.000000,3.136364,1.363636,0.318182
4,Kyle Connor,0.153,0.350000,3,0,1,0,1,0,0.296296,0.592593,0.222222,0.074074,0.0,0.000000,4.370370,0.074074,0.259259
